# Understanding delegates

A delegate encapsulates a method.

## Actions

`System.Action` is the standard library type for void delegates, methods that don't return a value. By default it's parameterless, but it can have parameters with generics.

In [ ]:
Action action = () => Console.WriteLine("Action done.");
// invoked as method call
action()

Action done.


In [ ]:
// user-declared delegate, no parameters
delegate void MyAction();
MyAction myAction = () => Console.WriteLine("My action done.");
myAction()

My action done.


I'm using lambdas here, which are anonymous functions that are defined on the fly, but delegates can accept already defined methods as well:

In [ ]:
/// Method that writes "Done." to console output.
public void WriteDone() {
    Console.WriteLine("Done.");
}

Action actionDone = WriteDone;
actionDone()

Done.


In [ ]:
Action actionWithParam = (int i) => Console.WriteLine(i);

Error: (1,26): error CS1593: Delegate 'Action' does not take 1 arguments

Declaring an `Action` with generic types to invoke it with parameters:

In [ ]:
// type of lambda parameter `i` is determined from declaration of delegate object
Action<int> actionOnInteger1 = i => Console.WriteLine(i);
// same as
// type of delegate object is determined from definition of lambda
var actionOnInteger2 = (int i) => Console.WriteLine(i);
actionOnInteger2(0)

0


In [ ]:
actionOnInteger2.GetType()

System.Action<System.Int32>

In [ ]:
// user-declared delegate, one parameter
delegate void MyActionOnInteger(int i);
MyActionOnInteger myActionOnInteger = i => Console.WriteLine(i);
myActionOnInteger(0)

0


In [ ]:
Action<int> actionThatReturns = i => { return i * 2; };

Error: (1,40): error CS8030: Anonymous function converted to a void returning delegate cannot return a value

Remember, `Action` is declared as a void delegate, it can't return any value.

## Funcs

`System.Func` is the standard library type for delegates that encapsulate methods that return a value. It can have parameters with generics too.

In [ ]:
// the last generic type is the return type. Parameters are optional
Func<string> func = () => "Function returned.";
var returned = func();
returned

Function returned.

In [ ]:
// user-declared delegate that returns a string, same as previous Func
delegate string MyFunc();
MyFunc myFunc = () => "My function returned.";
returned = myFunc();
returned

My function returned.

In [ ]:
Func<int, int, int> funcMultiply = (x, y) => x * y;
funcMultiply(2, 3)

6

In [ ]:
delegate int MyFuncMultiply(int x, int y);
MyFuncMultiply myFuncMultiply = (x, y) => x * y;
myFuncMultiply(2, 3)

6

## Use cases

- There's no need to declare new custom delegates, use `Action` and `Func` types.
- You can just use `var` and declare the types in the lambda definition. So why being aware of delegate types?:

### Delegates as abstractions

Code calling a delegate used as an abstraction doesn't need to know (and doesn't care) about how that delegate is implemented.

In [ ]:
public void WriteAsUpper(string message) {
    Console.WriteLine(message.ToUpper());
}

public void WriteLength(string message) {
    Console.WriteLine(message.Length);
}

public string CreateMessage(string title, string text, DateTime date, Action<string> writeCallback) {
    var message = $"{title}\n{text}\n{date.ToLongDateString()}";

    // I already know the delegate type: it receives a string and doesn't return anything. Ok more than enough info
    // But what does it do? I COULD CARE LESS, just call it :)
    writeCallback(message);
    
    return message;
}

In [ ]:
var redMessage = CreateMessage("Red", "Roses are red", DateTime.Now, WriteLength);

44


In [ ]:
redMessage

Red
Roses are red
Saturday, January 15, 2022

In [ ]:
var blueMessage = CreateMessage("Blue", "Blue roses are fake", DateTime.Now, WriteAsUpper);

BLUE
BLUE ROSES ARE FAKE
SATURDAY, JANUARY 15, 2022


In the above example, `CreateMessage` was defined to create a message and return it. It wasn't defined thinking about the console. If this method could speak it would tell you:
> If you want to show the message in console, create a delegate to do it and pass it to me as the `callback` parameter. Or whatever you want to do with it just create a delegate for that and pass it. I don't care, that's the callback's job.